# SQL Intermediate

In [ ]:
# Run this cell to load the tdquiz magic command, that runs queries and return the results.
%load_ext tdquiz

# Test tdquiz magic command
%tdquiz SELECT USER, SESSION, CURRENT_TIMESTAMP

<h3>Elements of date</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">sys_calendar</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">current_timestamp</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">extract</span>
<br>
For the current in Japan (UTC+9), show the date, day of the week, year, month, day, and hour.
Note that we want the day of week to be shown in Japanese here:
Monday="月曜日", Tuesday="火曜日", Wednesday="水曜日", Thursday="木曜日", Friday="金曜日", Saturday="土曜日", and Sunday="日曜日".
<details><summary>Hint</summary><ul>
<li><a href="https://www.docs.teradata.com/r/Teradata-VantageTM-SQL-Date-and-Time-Functions-and-Expressions-17.20/DateTime-and-Interval-Functions-and-Expressions/EXTRACT" rel="noopener noreferrer" target="_blank">EXTRACT</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">with dow as (
  /*現在の日付と曜日を取得*/
  select 
    calendar_date
    ,case day_of_week
    when 1 then '日曜日'
    when 2 then '月曜日'
    when 3 then '火曜日'
    when 4 then '水曜日'
    when 5 then '木曜日'
    when 6 then '金曜日'
    when 7 then '土曜日'
    else null end as "曜日"
    ,current_timestamp AT TIME ZONE 'GMT+9' as timestamp_jst
  from sys_calendar.calendar
  where calendar_date = current_date
)
select
  /*現在の日付*/
  TRUNC(timestamp_jst, 'DD') AS dt
  /*現在の曜日*/
  ,a1."曜日"
  /*タイムスタンプを年・月・日・時に分解*/
  ,extract(year from timestamp_jst) as "year"
  ,extract(month from timestamp_jst) as "month"
  ,extract(day from timestamp_jst) as "day"
  ,extract(hour from timestamp_jst) as "hour"
from dow a1;</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Calculation of Age</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">date</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">cast</span>
<br>
The <em>Terashirt.customer</em> table contains data on Terashirt members' birthdays and membership registration dates.
For each member, calculate the member's age at registration and current age.
Note that if the birthday is missing, set the age as missing (null) as well.<details><summary>Hint</summary>What would happen if you cast a DATE variable to INTEGER?</details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  customer_id
  ,floor((cast(register_date as integer) + 19000000) - (cast(birthday as integer) + 19000000)) / 10000 as "会員登録時年齢"
  ,floor((cast(current_date as integer) + 19000000) - (cast(birthday as integer) + 19000000)) / 10000 as "現在年齢"
FROM
  TeraShirt.customer
ORDER BY 1;</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Obtaining employee salary rankings</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span>
<br>
Find the employees with the largest and smallest amonts of salaries, and display their ranking, last name, first name and the salary amounts.
In case of ties, display both employees.
<details><summary>Hint</summary><ul>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Functions-Expressions-and-Predicates-17.20/Ordered-Analytical/Window-Aggregate-Functions/ROW_NUMBER" rel="noopener noreferrer" target="_blank">ROW_NUMBER</a>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Functions-Expressions-and-Predicates-17.20/Ordered-Analytical/Window-Aggregate-Functions/FIRST_VALUE/LAST_VALUE" rel="noopener noreferrer" target="_blank">FIRST_VALUE/LAST_VALUE</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  rank() OVER (ORDER BY salary DESC) AS ranking,
  last_name,
  first_name,
  salary
FROM
  TeraShirt.employee
QUALIFY
  rank() OVER (ORDER BY salary DESC) = 1
  OR rank() OVER (ORDER BY salary) = 1</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>First customer</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span>
<br>
For each store of the TeraShirt Company, fund the first customer who made a purchase in it.
Display the store name, purchase datetime, and the customer name (last name and first name with a space inbetween).
For the convenience, ignore the records where the customer name is missing.
If there are multiple more than one customers who made purchases at the same store and exactly the same time, then show them all.
<details><summary>Hint</summary><ul>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Functions-Expressions-and-Predicates-17.20/Ordered-Analytical/Window-Aggregate-Functions/ROW_NUMBER" rel="noopener noreferrer" target="_blank">ROW_NUMBER</a>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Functions-Expressions-and-Predicates-17.20/Ordered-Analytical/Window-Aggregate-Functions/FIRST_VALUE/LAST_VALUE" rel="noopener noreferrer" target="_blank">FIRST_VALUE/LAST_VALUE</a>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Data-Manipulation-Language-17.20/SELECT-Statements/QUALIFY-Clause" rel="noopener noreferrer" target="_blank">QUALIFY Clause</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT DISTINCT
  str.name,
  s.purchase_timestamp,
  c.last_name || '  '|| c.first_name AS customer_name
FROM
  TeraShirt.store_sales AS s
  INNER JOIN TeraShirt.store AS str
    ON s.store_id = str.store_id
  INNER JOIN TeraShirt.customer AS c
    ON s.customer_id = c.customer_id
QUALIFY
  purchase_timestamp = MIN(purchase_timestamp) OVER (PARTITION BY str.name)</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Salary difference from the average</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">aggregation</span>
<br>
The <em>Terashirt.employee</em> table contains salary data for Terashirt employees.
Calculate the overall salary average and the salary average per position, and for each employee calculate the difference from the overall salary average and the difference from the salary average per position.
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">select
  last_name || ' ' || first_name AS name
  ,"role"
  ,salary
  ,avg(salary) over() as "全体給与平均"
  ,avg(salary) over(partition by "role") as "roleごと給与平均"
  ,salary - "全体給与平均" as "全体給与平均との差額"
  ,salary - "roleごと給与平均" as "roleごと給与平均との差額"
from TeraShirt.employee
order by 6,7;</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Smoothing revenues at stores</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">moving average</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">sys_calendar</span>
<br>
TeraShirt company tracks the store revenues on the daily basis, but the revenue values may contain noise and seasonal variations.
To overcome this limitation, they have decided to include the moving average to the report.
<br />
For each store and date, calculate the total sales revenue and the average from three days before to three days after (i.e. moving average for the seven days window).
Display the store id, store name, date, revenue and the moving average.
<br />
For simplicitly, report only for the thirty days in November 2022, and make sure to include the days with no revenue.
<br />
Order the result by the store id and the date.

<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">WITH rev AS (
  SELECT
    store_id,
    purchase_date,
    SUM(sales_value) AS revenue
  FROM
    TeraShirt.store_sales
  GROUP BY 1,2
)
,alldays AS (
  SELECT
    s.store_id, c.calendar_date
  FROM
    sys_calendar.calendar AS c,
    (SELECT DISTINCT store_id FROM TeraShirt.store_sales) AS s
  WHERE
    c.calendar_date BETWEEN DATE '2022-10-29' AND DATE '2022-12-03'
)
,ma AS (
  SELECT 
    c.store_id,
    c.calendar_date,
    COALESCE(a.revenue, 0) AS revenue,
    AVG(COALESCE(a.revenue, 0)) OVER (
      PARTITION BY c.store_id ORDER BY c.calendar_date
      ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS revenue_moving_average
  FROM
    alldays AS c
    LEFT JOIN rev AS a
      ON c.store_id = a.store_id AND c.calendar_date = a.purchase_date
)
SELECT
  ma.store_id,
  s.name,
  ma.calendar_date,
  ma.revenue,
  ma.revenue_moving_average
FROM
  ma INNER JOIN TeraShirt.store AS s ON ma.store_id = s.store_id
ORDER BY 1,3</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>The longest time difference from the previous purchase</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">lag</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">interval</span>
<br>
In the sales record of the TeraShirt company, find the record with the longest duration from the previous purchase by the same cusomter.
Report the customer name (last name and first name concatenated by a space), purchase timestamp, previous purchase timestamp, and the time difference.
Express the time difference in hour unit up to the second decimal place.
Ignore the records with missing customer name in this analysis.
<details><summary>Hint</summary><ul>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Functions-Expressions-and-Predicates-17.20/Ordered-Analytical/Window-Aggregate-Functions/LAG/LEAD" rel="noopener noreferrer" target="_blank">LAG/LEAD</a>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Date-and-Time-Functions-and-Expressions/March-2019/DateTime-and-Interval-Functions-and-Expressions/Arithmetic-Operators-and-ANSI-DateTime-and-Interval-Data-Types/Calculating-the-Difference-Between-Two-DateTime-Values" rel="noopener noreferrer" target="_blank>Calculating the Difference Between Two DateTime Values</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">WITH tmp AS (
  SELECT
    c.last_name || ' ' || c.first_name AS customer_name,
    s.purchase_timestamp,
    lag(s.purchase_timestamp) OVER (PARTITION BY s.customer_id ORDER BY purchase_timestamp) AS prev_purchase_timestamp,
    (s.purchase_timestamp - prev_purchase_timestamp) DAY(4) TO HOUR AS time_diff
  FROM
    TeraShirt.store_sales AS s
    INNER JOIN TeraShirt.customer AS c
      ON s.customer_id = c.customer_id
  QUALIFY prev_purchase_timestamp IS NOT NULL
)
SELECT
  customer_name, purchase_timestamp, prev_purchase_timestamp,
  EXTRACT(DAY FROM time_diff) + 1.00*EXTRACT(HOUR FROM time_diff)/24 AS day_diff
FROM
  tmp
QUALIFY time_diff = MAX(time_diff) OVER ()</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Histogram</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">width_bucket</span>
<br>
The <em>Terashirt.store_sales</em> table contains historical product purchase data for each customer.
After calculating the total amount of money spent per purchase for each customer, we want to create a histogram of the amount of money spent, with 10 classes.
In this case, find the minimum and maximum number of frequencies (number of data) included in each class and the minimum and maximum amount of salary included in each class.
Note that data with null customer IDs should be excluded from the total.<details><summary>Hint</summary><ul>
<li><a href="https://docs.teradata.com/r/Teradata-VantageTM-SQL-Functions-Expressions-and-Predicates-17.20/Arithmetic-Trigonometric-Hyperbolic-Operators/Functions/WIDTH-BUCKET" rel="noopener noreferrer" target="_blank">WIDTH_BUCKET</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">with src0 as (
  select
  customer_id
  ,purchase_date
  ,purchase_timestamp
  ,sum(sales_value) as total_sales_value
  from terashirt.store_sales 
  where customer_id is not null
  group by 1,2,3
)
,src1 as (
  select
  max(total_sales_value)+1 as max_sales_value
  ,min(total_sales_value) as min_sales_value
  ,max_sales_value - min_sales_value as range_sales_value
  ,10 as bucket_num
  from src0
)
,src2 as (
  select
  total_sales_value
  ,width_bucket(total_sales_value, min_sales_value, max_sales_value, bucket_num) as bucket
  ,count(total_sales_value) as cntnum
  from src0
  ,src1
  group by 1  
)
select
bucket
,min(total_sales_value) as lower_sales_value
,max(total_sales_value) as upper_sales_value
,sum(cntnum) as "度数"
from  src2
group by 1
order by 1;</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Date Calculation</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">date</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">trunc</span>
<br>
Get the first day/last day of the current month, the first day/last day of the previous month,
and the first day/last day of the current month one year ago.<details><summary>Hint</summary><ul>
<li><a href="https://www.docs.teradata.com/r/Teradata-VantageTM-SQL-Date-and-Time-Functions-and-Expressions-17.20/DateTime-and-Interval-Functions-and-Expressions/TRUNC-Date" rel="noopener noreferrer" target="_blank">TRUNC(date)</a>
</ul></details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">select
trunc(current_date, 'month') as "当月初日"
,"当月初日" + interval '1' month - interval '1' day as "当月最終日"
,"当月初日" - interval '1' month as "前月初日" 
,"当月初日" - interval '1' day as "前月最終日"
,"当月初日" - interval '12' month as "前年当月初日"
,"当月初日" - interval '11' month - interval '1' day as "前年当月最終日" ;</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Recent Holidays</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">date</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">filtering</span> <span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">performance</span>
<br>
<em>japan.japan_holidays</em> table stores the Japan's national holidays since 1955.
You are trying to display the list of all holidays in the past 50 days, including today.
To do so, you have submitted the following query:
<pre class="sql-code">
SELECT
  calendar_date, holiday_name
FROM
  japan.japan_holidays
WHERE
  CAST(CURRENT_TIMESTAMP AS DATE) - calendar_date BETWEEN 0 AND 49
</pre>
While this query returns the correct outcome, it is not ideal for the query performance.
Please submit a more efficient query that returns the identical output.
<details><summary>Hint</summary>Try <code>SHOW TABLE</code> and check the DDL of the target table.
There might be some hit for improving the efficiency.</details>
<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">SELECT
  calendar_date, holiday_name
FROM
  japan.japan_holidays
WHERE
  calendar_date BETWEEN
    CAST(CURRENT_TIMESTAMP AS DATE) - INTERVAL '49' DAY AND
    CAST(CURRENT_TIMESTAMP AS DATE)</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Trend of the registered customers</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span>
<br>
Using the customer list of the TeraShirt company, calculate the trend of the number of the registered customers.
<br />
Output columns are the date and the number of customers as of that date (includes those registerd on that day).
<br />
Display only the days when the number of customers changed, and sort them by the date.

<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">WITH tmp AS (
  SELECT register_date, COUNT(*) AS n_inc
  FROM TeraShirt.customer
  GROUP BY 1
)
SELECT
  register_date AS "date",
  SUM(n_inc) OVER (
    ORDER BY register_date
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS num_registered_customers
FROM
  tmp</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */






<h3>Trend of cumulative sales</h3><span class="tag" style="font-size: smaller; background-color: #dddddd; color: #222222">window-function</span>
<br>
Using the store sales of the TeraShirt company, calculate the cumulative sales values by year at the end of each month.
<br />
Display year, date (the last day of the month), and cumulative sales values and sort them by the date.

<details><summary>Answer</summary><pre style="margin: 1em 0; padding: 1em; border-radius: 5px; background-color: #25292f; white-space: pre-wrap;"><code style="background-color: #25292f; color: #ffffff">WITH tmp AS (
  SELECT
    EXTRACT(YEAR FROM purchase_date) AS "year",
    TRUNC(purchase_date, 'MM') + 
      INTERVAL '1' MONTH - INTERVAL '1' DAY AS month_end,
    SUM(sales_value) AS total_sales
  FROM
    TeraShirt.store_sales
  GROUP BY 1,2
)
SELECT
  "year", month_end,
  SUM(total_sales) OVER (
    PARTITION BY "year"
    ORDER BY month_end
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS cum_sales
FROM
  tmp
ORDER BY 2</code></pre></details>

In [ ]:
%%tdquiz
/* Write your query blow */




